# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. d. Brok  ->  J. d. Brok  |  ['J. d. Brok']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
T. Henning  ->  T. Henning  |  ['T. Henning']
P. Boley  ->  P. Boley  |  ['P. Boley']


M. Zhang  ->  M. Zhang  |  ['M. Zhang']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
B. Pennell  ->  B. Pennell  |  ['B. Pennell']
J. Li  ->  J. Li  |  ['J. Li']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']
G. Rouillé  ->  G. Rouillé  |  ['G. Rouillé']
Arxiv has 82 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.05214


extracting tarball to tmp_2510.05214...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2510.05214/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'table_sample' from 'tmp_2510.05214/table_sample.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors' from 'tmp_2510.05214/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_d

...

J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
B. Pennell  ->  B. Pennell  |  ['B. Pennell']
J. Li  ->  J. Li  |  ['J. Li']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/orbit_fit.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth,angle=0,clip=true]{figures/orbit_fit.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/plot_orbit_6022870954609549824.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth]{figures/plot_orbit_6022870954609549824.png}
  warnings.warn(LatexWarn

Found 44 bibliographic references in tmp_2510.05982/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2510.06167


extracting tarball to tmp_2510.06167...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.05818-b31b1b.svg)](https://arxiv.org/abs/2510.05818) | **An interferometric mid-infrared study of the eruptive star binary Z CMa with MATISSE/VLTI. I. Imaging the protoplanetary disks during the 2023 outburst**  |
|| F. Lykou, et al. -- incl., <mark>T. Henning</mark>, <mark>P. Boley</mark> |
|*Appeared on*| *2025-10-08*|
|*Comments*| *25 pages, 21 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            The mid-infrared (MIR) emitting regions of the individual protoplanetary disks in the binary system Z CMa are resolved by MATISSE/VLTI. The observations were obtained during a serendipitous large outburst of the HBe star that lasted more than 100 days, while the FUor companion is presumed to be in quiescence. The size of the MIR-emitting disk region of the more massive HBe star increases toward longer wavelengths from $<14$ mas at 3.5$\mu m$ to $\ll 50$ mas at 11.5$\mu m$ . The lack of substructures in the HBe disk might suggest that it is a continuous disk; however, this could be due to observational constraints. We also note a radial variation of the silicate absorption feature over the disk, where the optical depth increases inwards of $<$40~au radii. This contradicts the scenario of a carved, dusty cocoon surrounding the HBe star. In the case of the less massive FUor companion, the MIR-emitting region is much smaller with an angular size $\leq$15 mas (or else a physical radius $<9$ au) in all bands, suggesting a compact disk. Both disks are aligned within uncertainties, and their orientation agrees with that of the known jets. Furthermore, MATISSE data place the binary's separation at $117.88 \pm 0.73$ mas and a position angle of $139.16^o\,\pm\,0.29^o$ east of north. Our estimates for the orbital elements gave an eccentric orbit ($e\sim0.17$) with a moderate inclination ($i\sim 66$\degr). The derived total mass is $M_{\rm total} = 16.4^{+2.1}_{-2.3}$ M$_\odot$, while the period is approximately 950 years. Our MATISSE imaging of the Herbig disk during outburst indicates a temperature gradient for the disk, while imaging of the FUor companion's disk corroborates previous studies showing that FUor disks are rather compact in the MIR. We cannot infer any misalignment between the MATISSE results and earlier ALMA/JVLA data, nor can we infer any influence from the alleged flyby event.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.05933-b31b1b.svg)](https://arxiv.org/abs/2510.05933) | **Magnetic Fields in the Bones of the Milky Way**  |
|| I. W. Stephens, et al. -- incl., <mark>M. Zhang</mark> |
|*Appeared on*| *2025-10-08*|
|*Comments*| *Accepted to ApJ*|
|**Abstract**|            Stars primarily form in galactic spiral arms within dense, filamentary molecular clouds. The largest and most elongated of these molecular clouds are referred to as ``bones," which are massive, velocity-coherent filaments (lengths ~20 to >100 pc, widths ~1-2 pc) that run approximately parallel and in close proximity to the Galactic plane. While these bones have been generally well characterized, the importance and structure of their magnetic fields (B-fields) remain largely unconstrained. Through the SOFIA Legacy program FIELDMAPS, we mapped the B-fields of 10 bones in the Milky Way. We found that their B-fields are varied, with no single preferred alignment along the entire spine of the bones. At higher column densities, the spines of the bones are more likely to align perpendicularly to the B-fields, although this is not ubiquitous, and the alignment shows no strong correlation with the locations of identified young stellar objects. We estimated the B-field strengths across the bones and found them to be ~30-150 $\mu$G at pc scales. Despite the generally low virial parameters, the B-fields are strong compared to the local gravity, suggesting that B-fields play a significant role in resisting global collapse. Moreover, the B-fields may slow and guide gas flow during dissipation. Recent star formation within the bones may be due to high-density pockets at smaller scales, which could have formed before or simultaneously with the bones.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.05982-b31b1b.svg)](https://arxiv.org/abs/2510.05982) | **Dormant BH candidates from Gaia DR3 summary diagnostics**  |
|| <mark>J. Müller-Horn</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>K. El-Badry</mark>, <mark>B. Pennell</mark>, <mark>J. Li</mark>, <mark>R. Seeburger</mark> |
|*Appeared on*| *2025-10-08*|
|*Comments*| *17 pages, 15 figures, submitted to A&A, comments welcome, catalogues are available here: this https URL*|
|**Abstract**|            We present a rigorous identification of candidates for dormant black holes (BHs) and neutron stars (NSs) in binaries using summary statistics from Gaia DR3, rather than full orbital solutions. Although Gaia astrometric orbits have already revealed a small sample of compact object binaries, many systems remain undetected due to stringent quality cuts imposed on the published orbits. Using a forward-modelling framework that simulates Gaia observables, in particular the renormalised unit weight error (ruwe) and radial velocity (RV) scatter, we infer posterior distributions for companion mass and orbital period via MCMC sampling, marginalising over nuisance orbital parameters. We validate our approach by comparing the predicted masses and periods against full orbit solutions from DR3, and by successfully recovering known compact object binaries as promising candidates. The method is best suited for systems with red giant primaries, which have more reliable Gaia RV scatter and a light centroid more likely dominated by one component, compared to main-sequence stars. And they are less likely to be triples with short-period inner binaries, which produce confounding signatures. We apply the method to three million giants and identify 556 systems with best-fit companion masses $\gtrsim 3\,M_\odot$. Recovery simulations suggest our selection method is substantially more sensitive than the DR3 non-single-star catalogue, particularly for binaries with periods below 1 year and above $\sim 6$ years. These candidates represent promising targets for spectroscopic follow-up and Gaia DR4 analysis to confirm the presence of compact objects. Candidate main-sequence stars with massive companions face a larger set of confounding effects. Therefore, we present an analogous catalogue of 279 additional `main sequence' candidates only as an appendix.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.05673-b31b1b.svg)](https://arxiv.org/abs/2510.05673) | **Resurging from the ashes: A spectral study of seven candidate revived radio fossils in nearby low-mass galaxy clusters**  |
|| L. Bruno, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-08*|
|*Comments*| *17 pages (12 main text + 5 appendix). Accepted for publication in A&A*|
|**Abstract**|            Complex energy transfer processes in the intracluster medium (ICM) can revive fossil (with spectral ages $\gg100$ Myr) plasma initially generated by radio galaxies. This leads to the re-ignition of faint radio sources with irregular and filamentary morphologies, and ultra-steep ($\alpha \gtrsim 1.5$) synchrotron spectra, which can be more easily detected at low frequencies ($\sim 100$ MHz). These sources offer the opportunity to investigate the microphysics of the ICM and its interplay with radio galaxies, the origin of seed relativistic electrons, the merging history of the host cluster, and the phenomenology of radio filaments. The study of revived sources has so far been hampered by the requirement of sensitive and high-resolution multi-frequency radio data at low frequencies to characterise their spatial properties and provide a proper classification. We aim to perform the analysis of a sample of candidate revived sources identified among nearby ($z\leq0.35$) and low-mass ($M_{500}\leq5\times 10^{14} M_\odot$) \textit{Planck} clusters in the footprint of LoTSS-DR2. By inspecting LoTSS-DR2 images at 144 MHz, we identified 7 targets with patchy and filamentary morphologies, which have been followed-up with the uGMRT at 400 MHz. By combining LOFAR and uGMRT data, we obtained high-resolution images and spectral index maps, which we used to interpret the nature of the sources. All targets show regions with very steep spectra, confirming the effectiveness of our morphology-based selection in identifying fossil plasma. Based on their morphology, spectral properties, and optical associations, we investigated the origin of the targets. We found a variety of promising revived fossil sources, while also showing that apparently intricate structures can be easily misclassified in the absence of high-resolution and multi-band data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.05587-b31b1b.svg)](https://arxiv.org/abs/2510.05587) | **Gaussian process analysis of type-B quasiperiodic oscillations in the black hole X-ray binary MAXI J1348-630**  |
|| <mark>Y. Wang</mark>, R. Ma, H. Zhang, D. Yan |
|*Appeared on*| *2025-10-08*|
|*Comments*| *Accepted for publication in A&A, 10 pages, 8 figures*|
|**Abstract**|            We analyzed Insight-HXMT data of the black hole X-ray binary MAXI J1348-630 during the type-B QPO phase of its 2019 outburst. Using the Gaussian process method, we applied an additive composite kernel model consisting of an SHO, a DRW, and an additional white noise (AWN) to data from three energy bands: LE (1-10 keV), ME (10-30 keV), and HE (30-150 keV). We find that for the DRW component, correlations on the timescale of $\tau_{\rm DRW}\sim10$ s are absent in the LE band, while they persist in the ME and HE bands over the full duration of the light curves. This energy-dependent behavior may reflect thermal instabilities, with the shorter correlation timescale in the disk compared to the corona. Alternatively, it may reflect variable Comptonizations of seed photons from different disk regions. Inner-disk photons are scattered by a small inner corona, producing soft X-rays. Outer-disk photons interact with an extended, jet-like corona, resulting in harder emission. The QPO is captured by an SHO component with a stable period of $\sim 0.2$ s and a high quality factor of $\sim 10$. The absence of significant evolution with energy or time of the SHO component suggests a connection between the accretion disk and the corona, which may be built by coherent oscillations of disk-corona driven by magnetorotational instability. The AWN components are present in all the three-band data and dominate over the DRW and SHO components. We interpret the AWN as another fast DRW with its $\tau_{\rm DRW} < 0.01$ s. It may trace high-frequency fluctuations that occur in both the inner region of the accretion disk and the corona. Overall, our work reveals a timescale hierarchy in the coupled disk-corona scenario: fast DRW < SHO < disk DRW < corona DRW.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.06167-b31b1b.svg)](https://arxiv.org/abs/2510.06167) | **PDRs4All XIX. The 6 to 9 $μ$m region as a probe of PAH charge and size in the Orion Bar**  |
|| B. Khan, et al. -- incl., <mark>G. Rouillé</mark> |
|*Appeared on*| *2025-10-08*|
|*Comments*| *13 pages, 10 figures, 2 tables. Submitted to A&A*|
|**Abstract**|            Infrared emission from polycyclic aromatic hydrocarbons (PAHs) play a major role in determining the charge balance of their host environments that include photo-dissociation regions (PDRs) in galaxies, planetary nebulae, and rims of molecular clouds. We aim to investigate the distribution and sizes of charged PAHs across the key zones of the Orion Bar PDR. We employ JWST MIRI-MRS observations of the Orion Bar from the Early Release Science program ''PDRs4All'' and synthetic images in the JWST MIRI filters. We investigate the spatial morphology of the AIBs at 6.2, 7.7, 8.6, and 11.0 $\mu$m that commonly trace PAH cations, and the neutral PAH-tracing 11.2 $\mu$m AIB, their (relative) correlations, and the relationship with existing empirical prescriptions for AIBs. The 6.2. 7.7, 8.6, 11.0, and 11.2 $\mu$m AIBs are similar in spatial morphology, on larger scales. Analyzing three-feature intensity correlations, two distinct groups emerge: the 8.6 and 11.0 $\mu$m vs. the 6.2 and 7.7 $\mu$m AIBs. We attribute these correlations to PAH size. The 6.2 and 7.7 $\mu$m AIBs trace cationic, medium-sized PAHs. Quantum chemical calculations reveal that the 8.6 $\mu$m AIB is carried by large, compact, cationic PAHs, and the 11.0 $\mu$m AIB's correlation to it implies, so is this band. The 6.2/8.6 and 7.7/8.6 PAH band ratios thus probe PAH size. We conclude that the 6.2/11.2 AIB ratio is the most reliable proxy for charged PAHs, within the cohort. We outline JWST MIRI imaging prescriptions that serve as effective tracers of the PAH ionization fraction as traced. This study showcases the efficacy of the 6-9 $\mu$m AIBs to probe the charge state and size distribution of the emitting PAHs, offering insights into the physical conditions of their host environments. JWST MIRI photometry offers a viable alternative to IFU spectroscopy for characterizing this emission in extended objects.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.05214-b31b1b.svg)](https://arxiv.org/abs/2510.05214) | **Resolved Profiles of Stellar Mass, Star Formation Rate, and Predicted CO-to-H$_2$ Conversion Factor Across Thousands of Local Galaxies**  |
|| J. Sun, et al. -- incl., <mark>J. d. Brok</mark>, <mark>A. Hughes</mark> |
|*Appeared on*| *2025-10-08*|
|*Comments*| *16 pages main text + 4 appendices. ApJ in press. We publish all data products (including galaxy sizes, UV and IR surface brightness profiles, stellar mass and SFR profiles, and conversion factor estimates) at this https URL*|
|**Abstract**|            We present radial profiles of surface brightness in UV and IR bands, estimate stellar mass surface density ($\Sigma_\star$) and star formation rate surface density ($\Sigma_\mathrm{SFR}$), and predict the CO-to-H$_2$ conversion factor ($\alpha_\mathrm{CO}$) for over 5,000 local galaxies with stellar mass $M_\star\,{\geq}\,10^{9.3}\rm\,M_\odot$. We build these profiles and measure galaxy half-light radii using GALEX and WISE images from the $z$0MGS program, with special care given to highly inclined galaxies. From the UV and IR surface brightness profiles, we estimate $\Sigma_\star$ and $\Sigma_\mathrm{SFR}$ and use them to predict $\alpha_\mathrm{CO}$ with state-of-the-art empirical prescriptions. We validate our (kpc-scale) $\alpha_\mathrm{CO}$ predictions against observational estimates, finding the best agreement when accounting for CO-dark gas as well as CO emissivity and excitation effects. The CO-dark correction plays a primary role in lower-mass galaxies, whereas CO emissivity and excitation effects become more important in higher-mass and more actively star-forming galaxies, respectively. We compare our estimated $\alpha_\mathrm{CO}$ to observed galaxy-integrated SFR to CO luminosity ratio as a function of $M_\star$. A large compilation of literature data suggests that star-forming galaxies with $M_\star = 10^{9.5{-}11}\,M_\odot$ show strong anti-correlations of SFR/$L^\prime_\mathrm{CO(1{-}0)} \propto M_\star^{-0.29}$ and SFR/$L^\prime_\mathrm{CO(2{-}1)} \propto M_\star^{-0.40}$. The estimated $\alpha_\mathrm{CO}$ trends, when combined with a constant molecular gas depletion time $t_\mathrm{dep}$, can only explain ${\approx}1/3$ of these SFR/$L^\prime_\mathrm{CO}$ trends. This suggests that $t_\mathrm{dep}$ being systematically shorter in lower-mass star-forming galaxies is the main cause of the observed SFR/$L^\prime_\mathrm{CO}$ variations. (Abridged)         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 41</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.05818/./figures/fig_01.png', 'tmp_2510.05818/./figures/fig_02.png', 'tmp_2510.05818/./figures/fig_04.png']
copying  tmp_2510.05818/./figures/fig_01.png to _build/html/
copying  tmp_2510.05818/./figures/fig_02.png to _build/html/
copying  tmp_2510.05818/./figures/fig_04.png to _build/html/
exported in  _build/html/2510.05818.md
    + _build/html/tmp_2510.05818/./figures/fig_01.png
    + _build/html/tmp_2510.05818/./figures/fig_02.png
    + _build/html/tmp_2510.05818/./figures/fig_04.png
found figures ['tmp_2510.05933/./vectors_Fil1_surfacedensity.png', '', 'tmp_2510.05933/./all_anglediff_vs_Ngas_2px_skip2.png', 'tmp_2510.05933/./all_cdfs_filPOS.png']
copying  tmp_2510.05933/./vectors_Fil1_surfacedensity.png to _build/html/
file not found 
copying  tmp_2510.05933/./all_anglediff_vs_Ngas_2px_skip2.png to _build/html/
copying  tmp_2510.05933/./all_cdfs_filPOS.png to _build/html/
exported in  _build/html/2510.05933.md
    + _build/html/tmp_2510.05933/./vectors_Fil1_

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\micron}{\rm \mum}$
$\newcommand{\massyear}{M_\odot~yr^{-1}}$
$\newcommand{\kms}{km s^{-1}}$</div>



<div id="title">

# An interferometric mid-infrared study of the eruptive star binary Z CMa with MATISSE/VLTI$\thanks{Reproduced with permission from Astronomy \& Astrophysics, \textcopyright  ESO.}$: I. Imaging the protoplanetary disks during the 2023 outburst$\thanks{Based on observations collected at the European Organisation for Astronomical Research in the Southern Hemisphere under ESO programs 0106.C-0501(B), 0108.C-0385(D), and 0110.C-4209(A).}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.05818-b31b1b.svg)](https://arxiv.org/abs/2510.05818)<mark>Appeared on: 2025-10-08</mark> -  _25 pages, 21 figures, accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

F. Lykou, et al. -- incl., <mark>T. Henning</mark>, <mark>P. Boley</mark>

</div>
<div id="abstract">

**Abstract:**            The mid-infrared (MIR) emitting regions of the individual protoplanetary disks in the binary system Z CMa are resolved by MATISSE/VLTI. The observations were obtained during a serendipitous large outburst of the HBe star that lasted more than 100 days, while the FUor companion is presumed to be in quiescence. The size of the MIR-emitting disk region of the more massive HBe star increases toward longer wavelengths from $<14$ mas at 3.5$\mu m$ to $\ll 50$ mas at 11.5$\mu m$ . The lack of substructures in the HBe disk might suggest that it is a continuous disk; however, this could be due to observational constraints. We also note a radial variation of the silicate absorption feature over the disk, where the optical depth increases inwards of $<$40~au radii. This contradicts the scenario of a carved, dusty cocoon surrounding the HBe star. In the case of the less massive FUor companion, the MIR-emitting region is much smaller with an angular size $\leq$15 mas (or else a physical radius $<9$ au) in all bands, suggesting a compact disk. Both disks are aligned within uncertainties, and their orientation agrees with that of the known jets. Furthermore, MATISSE data place the binary's separation at $117.88 \pm 0.73$ mas and a position angle of $139.16^o\,\pm\,0.29^o$ east of north. Our estimates for the orbital elements gave an eccentric orbit ($e\sim0.17$) with a moderate inclination ($i\sim 66$\degr). The derived total mass is $M_{\rm total} = 16.4^{+2.1}_{-2.3}$ M$_\odot$, while the period is approximately 950 years. Our MATISSE imaging of the Herbig disk during outburst indicates a temperature gradient for the disk, while imaging of the FUor companion's disk corroborates previous studies showing that FUor disks are rather compact in the MIR. We cannot infer any misalignment between the MATISSE results and earlier ALMA/JVLA data, nor can we infer any influence from the alleged flyby event.         

</div>

<div id="div_fig1">

<img src="tmp_2510.05818/./figures/fig_01.png" alt="Fig7" width="100%"/>

**Figure 7. -** $V$-band light curve of Z CMa with corresponding epochs of MATISSE observations (orange lines; dashed and solid for the quiescent and the outburst phase, respectively) as in Table \ref{tab:matlog}. Photometric data from the Archive of the American Association of Variable Star Observers (AAVSO; green circles), the INTErnational Gamma-Ray Astrophysics Laboratory (INTEGRAL) Optical Monitoring Camera (OMC; magenta circles), and the Kamogata/Kiso/Kyoto Wide-field Survey of Variable Star Observers League in Japan \citep[KWS; blue circles][]{vsolj}. The imaging data were obtained during the outburst from late 2022 to early 2023. Also shown for reference are the epochs of archival ALMA and JVLA observations \citep[purple lines;][]{takami2019}. (*fig:vislc*)

</div>
<div id="div_fig2">

<img src="tmp_2510.05818/./figures/fig_02.png" alt="Fig1" width="100%"/>

**Figure 1. -** Dereddened SED of Z CMa during the 2023 outburst. Our own SpeX/IRTF and MATISSE spectra are shown in red and blue, respectively. For a description of the archival photometry (black points), we refer the reader to Sect. \ref{phot}. A Kurucz model (green) and a blackbody model (orange) are shown for reference.  (*fig:specsed*)

</div>
<div id="div_fig3">

<img src="tmp_2510.05818/./figures/fig_04.png" alt="Fig8" width="100%"/>

**Figure 8. -** MiRA image reconstructions in $L$, $M$, and $N$ bands during the 2023 outburst. From left to right and top to bottom, these correspond to pseudo-continuum reconstructed images at 3.5, 4.7, 8.5, 9.2, 10.4, and 11.5 $\micron$. These are smoothed with a Gaussian beam with a FWHM similar to the image resolution, normalized to unity, and shown in a logarithmic stretch with a minimum value of $10^{-2}$ to suppress image reconstruction residuals below that level. Such residuals are the result of insufficient $uv$ coverage (Fig. \ref{fig:uv}).  The SE component (FUor) becomes much fainter than the NW component (HBe) at longer wavelengths. The beam size is shown as a black circle at the bottom left corner of each panel.  (*fig:mirarecon*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.05818"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pfrac}{P_{\text{frac}}}$
$\newcommand{\sigpfrac}{\sigma_{P_{\text{frac}}}}$
$\newcommand{\ceooz}{\mbox{C^{18}O(1--0)}}$
$\newcommand{\ceoto}{\mbox{C^{18}O(2--1)}}$
$\newcommand{\ttcooz}{\mbox{^{13}CO(1--0)}}$
$\newcommand{\ttcoto}{\mbox{^{13}CO(2--1)}}$
$\newcommand{\Ngas}{N_{\text{gas}}}$
$\newcommand{\smallngas}{n_{\text{gas}}}$
$\newcommand{\NHt}{N_{\text{H_2}}}$
$\newcommand{\NHe}{N_{\text{He}}}$
$\newcommand{\muHt}{\mu_{\text{H_2}}}$
$\newcommand{\mup}{\mu_p}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\tco}{\mbox{^{13}CO(1--0)}}$
$\newcommand{\ceo}{\mbox{C^{18}O(1--0)}}$
$\newcommand{\nht}{\mbox{NH_3(1,1)}}$</div>



<div id="title">

# Magnetic Fields in the Bones of the Milky Way

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.05933-b31b1b.svg)](https://arxiv.org/abs/2510.05933)<mark>Appeared on: 2025-10-08</mark> -  _Accepted to ApJ_

</div>
<div id="authors">

I. W. Stephens, et al. -- incl., <mark>M. Zhang</mark>

</div>
<div id="abstract">

**Abstract:** Stars primarily form in galactic spiral arms within dense, filamentary molecular clouds. The largest and most elongated of these molecular clouds are referred to as "bones," which are massive, velocity-coherent filaments (lengths $\sim$ 20 to $>$ 100 pc, widths $\sim$ 1--2 pc) that run approximately parallel and in close proximity to the Galactic plane. While these bones have been generally well characterized, the importance and structure of their magnetic fields (B-fields) remain largely unconstrained. Through the SOFIA Legacy program FIELDMAPS, we mapped the B-fields of 10 bones in the Milky Way. We found that their B-fields are varied, with no single preferred alignment along the entire spine of the bones. At higher column densities, the spines of the bones are more likely to align perpendicularly to the B-fields, although this is not ubiquitous, and the alignment shows no strong correlation with the locations of identified young stellar objects. We estimated the B-field strengths across the bones and found them to be $\sim$ 30--150 $\mu$ G at pc scales. Despite the generally low virial parameters, the B-fields are strong compared to the local gravity, suggesting that B-fields play a significant role in resisting global collapse. Moreover, the B-fields may slow and guide gas flow during dissipation. Recent star formation within the bones may be due to high-density pockets at smaller scales, which could have formed before or simultaneously with the bones.

</div>

<div id="div_fig1">

<img src="tmp_2510.05933/./vectors_Fil1_surfacedensity.png" alt="Fig1.1" width="50%"/><img src="" alt="Fig1.2" width="50%"/>

**Figure 1. -** 
Top Panel: Filament 1 inferred B-field vectors overlaid on the Herschel-derived column density map, with contours for this figure and in Appendix \ref{appendix:vectormaps} at [0.63, 0.75, 1.3, 1.8, 2.5, 5.0, 7.5]$\times$ 10$^{22}$ cm$^{-2}$. Bright red and yellow vectors have measurements of $\pfrac$/$\sigpfrac$$>$ 3 and 2 $\leq$\pfrac/$\sigpfrac$$\leq$ 3, respectively; faded vectors indicate where $\pfrac$$>$ 0.15. Blue and green circles mark known Class I and II YSOs, respectively (Section \ref{sec:YSOs}).  Vectors are spaced by 2 HAWC+ pixels (9$\farcs$1), near Nyquist sampling. Bottom Panel: LIC map overlaid on column density, where the wavy pattern shows the B-field direction for $\pfrac$/$\sigpfrac$$>$ 1.5 and $I/\sigma_I$$>$ 10. The spine's polynomial fit is the solid line. The white beam (top right;  36$\farcs$4) shows the resolution for the column density map and the orange beam (bottom right; 18$\farcs$7) shows the resolution for SOFIA/HAWC+.  (*fig:fil1vectors*)

</div>
<div id="div_fig2">

<img src="tmp_2510.05933/./all_anglediff_vs_Ngas_2px_skip2.png" alt="Fig4" width="100%"/>

**Figure 4. -** $\Ngas$ versus $|\theta_B -\theta_{\text{fil}}|$ for each bone, shown for every other HAWC+ pixel (approximately Nyquist sampled). Only data within 23$\arcsec$ of the bone (two spine pixels) are considered. The data is binned into 10 equal $\Ngas$ ranges, and box plots are shown if there are at least 4 points in the bin. The box plots show the first and third quartiles in black and the median in orange. A red vertical line is shown for the first $\Ngas$ box plot bin where $|\theta_B -\theta_{\text{fil}}|$$>$ 60$^\circ$, which indicates the first sign of perpendicularity with $\Ngas$. Blue and green circles show the $\Ngas$ and $|\theta_B -\theta_{\text{fil}}|$ values at the nearest pixel for the known Class I and II YSOs, respectively.  (*fig:columndiff*)

</div>
<div id="div_fig3">

<img src="tmp_2510.05933/./all_cdfs_filPOS.png" alt="Fig5" width="100%"/>

**Figure 5. -** Cumulative distribution function of the angle difference between each bone's spine and B-field direction, $|\theta_B -\theta_{\text{fil}}|$. Only pixels within 23$\arcsec$ of the spine are considered. The red line in each panel shows the results for that particular bone. The blue lines in each panel show a Monte Carlo simulation of the expected distribution of projected angles if the spine and B-field directions in 3D are mostly parallel (within $0^\circ$ to $20^\circ$; dashed blue line), mostly perpendicular (within $70^\circ$ to $90^\circ$; dashed-dotted blue line), or random (solid blue line). The cyan lines show the expected projected angles if the bone is fixed in the plane-of-the-sky for mostly parallel (dashed cyan line) and mostly perpendicular (dashed-dotted cyan line).   (*fig:cdfs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.05933"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\aj}{Astronomical Journal}$
$\newcommand{\apj}{Astrophysical Journal}$
$\newcommand{\aap}{Astronomy \& Astrophysics}$
$\newcommand{\jcap}{Journal of Cosmology and Astroparticle Physics}$
$\newcommand{\aapr}{The Astronomy and Astrophysics Review}$
$\newcommand{\mnras}{Monthly Notices of the Royal Astronomical Society}$
$\newcommand{\araa}{Annual Review of Astronomy and Astrophysics}$
$\newcommand{\Msun}{\mathrm{M}_\odot}$</div>



<div id="title">

# Dormant BH candidates from _Gaia_ DR3 summary diagnostics

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.05982-b31b1b.svg)](https://arxiv.org/abs/2510.05982)<mark>Appeared on: 2025-10-08</mark> -  _17 pages, 15 figures, submitted to A&A, comments welcome, catalogues are available here: this https URL_

</div>
<div id="authors">

<mark>J. Müller-Horn</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>K. El-Badry</mark>, <mark>B. Pennell</mark>, <mark>J. Li</mark>, <mark>R. Seeburger</mark>

</div>
<div id="abstract">

**Abstract:** We present a rigorous identification of candidates for dormant black holes (BHs) and neutron stars (NSs) in binaries using summary statistics from _Gaia_ Data Release 3 (DR3), rather than full orbital solutions. Although _Gaia_ astrometric orbits have already revealed a small sample of compact object binaries, many systems remain undetected due to stringent quality cuts imposed on the published orbits.    Using a forward-modelling framework that simulates _Gaia_ observables, in particular the renormalised unit weight error ( \texttt{ruwe} ) and radial velocity (RV) scatter, we infer posterior distributions for companion mass and orbital period via Markov chain Monte Carlo (MCMC) sampling, marginalising over nuisance orbital parameters. We validate our approach by comparing the predicted masses and periods against full orbit solutions from DR3, and by successfully recovering known compact object binaries as promising candidates.    The method is best suited for systems with red giant primaries, which have more reliable _Gaia_ RV scatter and a light centroid more likely dominated by one component, compared to main-sequence stars. And they are less likely to be triples with short-period inner binaries, which produce confounding signatures.    We apply the method to three million giants and identify 556 systems with best-fit companion masses $\gtrsim 3 \mathrm{M}_\odot$ . Recovery simulations suggest our selection method is substantially more sensitive than the DR3 non-single-star catalogue, particularly for binaries with periods below 1 year and above $\sim 6$ years.    These candidates represent promising targets for spectroscopic follow-up and _Gaia_ DR4 analysis to confirm the presence of compact objects and build a more complete census of the Galactic BH and NS population. Candidate main-sequence stars with massive companions face a larger set of confounding effects. Therefore, we present an analogous catalogue of 279 additional `main sequence' candidates only as an appendix.

</div>

<div id="div_fig1">

<img src="tmp_2510.05982/./figures/sample_completeness.png" alt="Fig16" width="100%"/>

**Figure 16. -** Simulated completeness of the RGB+BH binary catalogue as a function of BH mass and orbital period. Top panels: Mean values of predicted \texttt{ruwe}(left) and \texttt{rv\_amplitude\_robust}(right) from mock _Gaia_ DR3 observations, shown for simulated binaries with BH masses of 5, 10, and 20 M$_\odot$. Shaded regions indicate one standard deviation from the mean. Threshold values for the cuts on excess scatter in RV and position applied to the candidate sample are shown in black. Larger BH masses generally yield stronger astrometric and RV signals. Astrometric \texttt{ruwe} increases with orbital period up to the DR3 baseline ($\sim$1000 d) while the RV amplitude is highest for short-period systems.
    Bottom panels: Recovery fractions for simulated binaries that pass the initial cuts on excess scatter in RV and position (left), that exceed the inferred companion mass threshold of 3 M$_\odot$(centre), and that meet both criteria (right). The final completeness is highest ($\gtrsim$80\% for 10–20 M$_\odot$ BH) for binaries with periods of 100–2000 d. Completeness decreases for longer periods due to incomplete phase coverage within the DR3 observing baseline. For reference, we show approximate completeness fractions for the _Gaia_ DR3 astrometric (dashed) and spectroscopic (dotted) orbital solutions estimated based on the \texttt{gaiamock} forward-model and SBX catalogue of spectroscopic binaries (see Sect. \ref{sec:completeness} for details). (*fig:completeness*)

</div>
<div id="div_fig2">

<img src="tmp_2510.05982/./figures/GaiaDR3_nss_hist2d_astrometric.png" alt="Fig2.1" width="50%"/><img src="tmp_2510.05982/./figures/GaiaDR3_nss_hist2d_spectroscopic_err.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Predicted vs. catalogued _Gaia_ DR3 summary statistics for known binaries, with the predictions based on forward modelling of their orbital parameters. Top panel: Comparison between predicted and catalogue values of the astrometric goodness of fit of the single-star astrometric solution, \texttt{ruwe}, for binaries with published orbital solutions. Bottom panel: Predicted vs. observed RV "errors" for spectroscopic binaries (SB1) in _Gaia_ DR3. In both plots, the blue line shows the median trend, while the dashed blue lines indicate the 16th and 84th percentiles. The 1:1 relation is marked in black. Lower subpanels show the logarithmic residuals between observed and predicted values, with side histograms illustrating their normalised distribution. The mean bias ($\mu$) and scatter ($\sigma$) of the residuals are indicated. (*fig:forward_model*)

</div>
<div id="div_fig3">

<img src="tmp_2510.05982/./figures/cleaning_fits_inferred_parameters_multipanel.png" alt="Fig6" width="100%"/>

**Figure 6. -** Inferred orbital periods and companion masses for the 1410 RGB stars with excess RV and astrometric scatter and companion mass estimates above $3 \mathrm{M}_\odot$. The top panel shows the final cleaned catalogue of 556 RGB+BH candidates (see Sect. \ref{sec:cleaning}), obtained after applying a series of quality cuts. The lower panels display the subsets of sources removed as likely contaminants by said cuts. Stars excluded because of spurious RV scatter (red) are concentrated at short inferred periods and high companion masses. Those with unreliable, underestimated parallaxes (pink) are typically found at long inferred periods. Eclipsing binaries and stars with photometric variability in excess of that expected from ellipsoidal modulation (purple) are scattered across both short and long periods. Systems excluded due to existing NSS orbital solutions (blue) occupy a broadly similar parameter space as the final candidates. (*fig:cleaning*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.05982"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

130  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

19  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
